# Urban Cooling using InVEST

**Data needs:**
1. Land use raster
2. Biophysical table (csv)
3. Reference evapotranspiration (raster)
4. Area of interest (vector)
5. Maximum cooling distance (m)
6. Reference air temperature (°C)
7. Urban Heat Index (UHI)
8. Air Blending distance (m)

**Initializing**

In [3]:
# Import modules
import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

In [6]:
# Set environment settings
arcpy.env.workspace = r"C:\Users\ABI\OneDrive - NIVA\PhD_Work\Work\PartII\Loren"

In [7]:
# Data imports
subcat = "C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\Data\\subcat_poly.shp"
landuse = Raster("C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\Data\\Landcover.tif")

**Pre-processing**

In [8]:
# Create boundary of shapefile
subcat_bdy = "C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\LorenGIS.gdb\\subcat_boundary"
arcpy.management.Dissolve(subcat, subcat_bdy,["FID"],"","SINGLE_PART","DISSOLVE_LINES")

<Result 'C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\LorenGIS.gdb\\subcat_boundary'>

Here, I am calculating the values for a buffer area of 100 m, which is an assumption of maximum cooling distance. The reason for doing this is that entities such as urban cooling is highly dependent on the surrounding few meters. There may be a difference in opinion in this case, since while calculating runoff, we are not consdering inflow runoff from the surrounding, instead we focus only an influx from the rainfall. However, I think the influence of the surrounding is much more prominent for these socioenvironmental variables, and they should be considereed. 
Cooling distance recommended by InVEST Mnaual is 450m , but (Zawadzka et al., 2021) found that this distance overestimates the influence of greenspaces in urban areas. Therefore, they recommend 100m.  Denser urban areas require shorter coolking distance.
This article also suggests that InVEST is not a good choice for small scale urban areas modelling. I will nevertheless be using it in this case. 

In [9]:
# Creating 400 m buffer of the boundary
subcat_buffer = "C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\LorenGIS.gdb\\subcat_buffer"
arcpy.analysis.Buffer(subcat_bdy, subcat_buffer,"100 Meters","FULL","ROUND","ALL")

<Result 'C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\LorenGIS.gdb\\subcat_buffer'>

**1.Landuse**

In [10]:
# Clipping landuse to buffer area
landuse_clipped = ExtractByMask(landuse, subcat_buffer )

# Save the output
#landuse_clipped.save("C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\LorenGIS.gdb\\landuse_clip")

**2. Biophysical Table**

*Crop coefficient (Kc):*
<br> Deep vegetation = 1 (Source: Kc calculator in InVEST Manual shows 1 to 1.2 for forests, FAO document)
<br> Shallow vegetation = 0.8 (Source: Assumed from InVEST Manual)
<br> Bare Soil = 0.2 (if actively growing, put higher)
<br> Unpaved road = 0.1
<br> Built areas = 0.001 (In InVEST Manual, recommended is to use f*0.1 +(1-f)*0.6, here impervious percent f is 0.72, so Kc is 0.24. This is a high value, so here a almost negligible value is used as also done by Zawadzka 2021.)


*Albedo*
<br> Buildings = 0.25 (Source: Zawadzka 2021)
<br> Shallow vegetation = 0.16 (Source: Zawadzka 2021)
<br> Deep vegetation = 0.2 (Source: Zawadzka 2021)
<br> Paved road = 0.14 (Source: Zawadzka 2021)
<br> Bare soil = 0.17 (Source: Markvart)
<br> Unpaved road = 0.2 
<br> Bare rock = 0.2 

In [11]:
# Create a dataframe for biophysiscal table
import pandas as pd
columns = ['lucode', 'Kc', 'green_area', 'shade', 'albedo']
data = [(1,0.2,0,0,0.17),(3,0.001,0,0,0.14), (6,0.8,1,0,0.16), (7,1,1,1,0.2), (9,0.001,0,0,0.14), (15,0.001,0,0,0.2), (16,0.001,0,0,0.25)]
df = pd.DataFrame(data, columns=columns)
print(df)
df.to_csv(r'C:\Users\ABI\OneDrive - NIVA\PhD_Work\Work\PartII\Loren\InVEST\UrbanCooling\BiophysicalTable.csv', index=False, sep = ';') 

   lucode     Kc  green_area  shade  albedo
0       1  0.200           0      0    0.17
1       3  0.001           0      0    0.14
2       6  0.800           1      0    0.16
3       7  1.000           1      1    0.20
4       9  0.001           0      0    0.14
5      15  0.001           0      0    0.20
6      16  0.001           0      0    0.25


**3. Reference evapotranspiration**

Reference evapotranspiration, ET₀, measures the amount of water that vaporizes from land into the air over a given period of time. It is typically expressed as a depth of water in millimeters per unit time.You can calculate reference ET by developing monthly average grids of precipitation, and maximum and minimum temperatures. A simple way to determine reference evapotranspiration is the ‘modified Hargreaves’ equation (Droogers and Allen, 2002), which generates superior results than the Pennman-Montieth when information is uncertain (InVEST Manual).
The ‘modified Hargreaves’ method uses the average of the mean daily maximum and mean daily minimum temperatures for each month (Tavg in degrees Celsius), the difference between mean daily maximum and mean daily minimums for each month (TD), extraterrestrial radiation (RA in MJm-2d-1) and average monthly precipitation (in mm per month), all of which can be relatively easily obtained.

Data was taken from the Nordic gridded temperature and precipitation data from cds.climate.copernicus.eu. Taking an average from many years and months would be better, but here for simplicity, the day with the highest temperature in 2023, June 27th, was taken as a reference. Only max, min temperatures, and precipitation grids from that day was extracted. 

The data is in the form of netcdf layer, that has to be first converted into a raster, projected to the current coordinate system, and then clipped to the boundary.

In [4]:
import os
import arcpy
from arcpy.sa import ExtractByMask

# Check out Spatial Analyst
arcpy.CheckOutExtension("Spatial")

# Boundary slightly larger than pixel
pixel_bdy = r"C:\Users\ABI\OneDrive - NIVA\PhD_Work\Work\PartII\Loren\LorenGIS.gdb\pixelboundary"

# Input NetCDFs
RR_nc = r"C:\Users\ABI\OneDrive - NIVA\PhD_Work\Work\PartII\Loren\Data\NGCD_RR_type2_version_24.09_20230627.nc"
TN_nc = r"C:\Users\ABI\OneDrive - NIVA\PhD_Work\Work\PartII\Loren\Data\NGCD_TN_type2_version_24.09_20230627.nc"
TX_nc = r"C:\Users\ABI\OneDrive - NIVA\PhD_Work\Work\PartII\Loren\Data\NGCD_TX_type2_version_24.09_20230627.nc"

# Output Geodatabase
gdb_path = r"C:\Users\ABI\OneDrive - NIVA\PhD_Work\Work\PartII\Loren\LorenGIS.gdb"

# Get projection from existing feature class
sr = arcpy.Describe(os.path.join(gdb_path, 'pixelboundary')).spatialReference

# Process each variable
for nc, var in zip((RR_nc, TN_nc, TX_nc), ("RR", "TN", "TX")):
    out_layer = f"{var}_layer"
    temp_raster = os.path.join(gdb_path, f"{var}_temp")
    proj_raster = os.path.join(gdb_path, f"{var}_proj")
    final_raster = os.path.join(gdb_path, f"{var}_masked")
    
    # Step 1: Create raster layer from NetCDF
    arcpy.md.MakeNetCDFRasterLayer(
        in_netCDF_file=nc,
        variable=var,
        x_dimension="X",
        y_dimension="Y",
        out_raster_layer=out_layer
    )
    
    # Step 2: Save raster layer as a permanent raster
    arcpy.management.CopyRaster(out_layer, temp_raster)
    
    # Step 3: Project raster to desired coordinate system
    arcpy.management.ProjectRaster(temp_raster, proj_raster, sr)

    # Step 4: Extract by pixel boundary using projected raster
    masked = ExtractByMask(proj_raster, pixel_bdy)
    
    # Step 5: Save final masked raster
    masked.save(final_raster)


Here, since it is a small area, the value is the same for the whole area. We still require a gridded data, therefore this whole operation was necessary.
Now, the modified Hargreaves’ equation (Droogers and Allen, 2002) as suggested by InVEST Manual was used to obtain the reference evapotranspiration.

In [10]:
# Get the rasters for further calculations
# The values 
P = Raster(os.path.join(gdb_path,'RR_masked'))                 # Precipitation
TN_ras = Raster(os.path.join(gdb_path,'TN_masked')) - 273      # Minimum daily temperature
TX_ras = Raster(os.path.join(gdb_path,'TX_masked')) - 273      #Maximum daily temperature

Tavg = (TN_ras + TX_ras)/2   # Tavg – the average of the daily minimum and daily maximum temperatures [°C]
TD = TX_ras - TN_ras         # TD – the difference between daily maximum and mean daily minimum temperatures [°C]
RA = 41.6                    # Extra-terrestrial radiation, estimated as 41.6 MJm-2d-1 (Zawadzka 2021)

# Reference evapotranspiration
ETo = 0.0013 * 0.408 * RA * (Tavg + 17) * (TD - 0.0123 * P)**0.76
ETo.save(os.path.join(gdb_path,'reference_ET'))

The reference evapotranspiration fro the study area was 5.03 mm/d.

**4. Area of interest**

In [12]:
AOI = "C:\\Users\\ABI\\OneDrive - NIVA\\PhD_Work\\Work\\PartII\\Loren\\LorenGIS.gdb\\subcat_buffer"

**5. Maximum cooling distance**

In [14]:
Max_cool_dist = 100 # Assumed to be 100 m for dense urban setting

**6. Reference air temperature**

Air temperature in a rural reference area where the urban heat island effect is not observed. A forested region above Årvoll was selected and the maximum temperature for that was around 24°C. 

In [16]:
Ref_air_temp = 24

**7. UHI Effect**

The magnitude of the urban heat island effect, i.e., the difference between the rural reference temperature and the maximum temperature observed in the city.

In [18]:
max_temp = 25.75
uhi = max_temp -  Ref_air_temp

**8. Air blending distance**

InVest Manual has suggested 500 to 600 m. Theses suggestions usually are for large scale regions, that are necessarily not urban areas. Since maximum cooling distance was decreased for urban areas according to the recommendation of Zawadazka 2021, the air blending distance is also assumed to be lesser than 500 m.

In [20]:
air_bl_dist = 300